In [27]:
# work in progress

# for experimental / educational purposes only

In [28]:
import sys
from pathlib import Path
PROJECT_DIR = Path.cwd()
if PROJECT_DIR.stem == 'notebooks':
    PROJECT_DIR = PROJECT_DIR.parent
    sys.path.insert(0, '..')

INTERACTIVE_PLOTTING = True

In [ ]:
# from https://www.datacamp.com/workspace/templates/playbook-python-multiple-time-series

In [ ]:
# Load packages
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics import tsaplots
import statsmodels.api as sm
import seaborn as sns

In [ ]:
# Upload your data as CSV and load as a data frame
df = pd.read_csv(
    "data.csv",
    parse_dates=["datestamp"],  # Tell pandas which column(s) to parse as dates
    index_col="datestamp",  # Use a date column as your index
)
df.head()


In [ ]:
# Plot settings
%config InlineBackend.figure_format='retina'
plt.rcParams["figure.figsize"] = (18, 10)
plt.style.use('ggplot')

# Plot all time series in the df DataFrame
ax = df.plot(
    colormap="Spectral",  # Set a colormap to avoid overlapping colors
    fontsize=10,  # Set fontsize
    linewidth=0.8, # Set width of lines
)

# Set labels and legend
ax.set_xlabel("Date", fontsize=12)  # X axis text
ax.set_ylabel("Unemployment Rate", fontsize=12) # Set font size
ax.set_title("Unemployment rate of U.S. workers by industry", fontsize=15)
ax.legend(
    loc="center left",  # Set location of legend within bounding box
    bbox_to_anchor=(1.0, 0.5),  # Set location of bounding box
)

# Annotate your plots with vertical lines
ax.axvline(
    "2001-07-01",  # Position of vertical line
    color="red",  # Color of line
    linestyle="--",  # Style of line
    linewidth=2, # Thickness of line
)
ax.axvline("2008-09-01", color="red", linestyle="--", linewidth=2)

# Show plot
plt.show()


In [ ]:
#  Generate a boxplot
ax = df.boxplot(fontsize=10, vert=False)  # Plots boxplot horizonally if false
ax.set_xlabel("Unemployment Percentage")
ax.set_title("Distribution of Unemployment by industry")
plt.show()

In [ ]:
# Display the autocorrelation plot of your time series
fig = tsaplots.plot_acf(
    df["Agriculture"], lags=24  # Change column to inspect
)  # Set lag period

# Show plot
plt.show()


In [ ]:
# Display the partial autocorrelation plot of your time series
fig = tsaplots.plot_pacf(
    df["Agriculture"], lags=24  # Change column to inspect
)  # Set lag period

# Show plot
plt.show()


In [ ]:
# Extract time period of interest
index_year = df.index.year  # Choose year, month, day etc.

# Compute mean for each time period
df_by_year = df.groupby(index_year).mean()  # Replace .mean() with aggregation function

# Plot the mean for each time period
ax = df_by_year.plot(fontsize=10, linewidth=1)

# Set axis labels and legend
ax.set_xlabel("Year", fontsize=12)
ax.set_ylabel("Mean unemployment rate", fontsize=12)
ax.axvline(
    2008,  # Position of vertical line
    color="red",  # Color of line
    linestyle="--",  # Style of line
    linewidth=2,
)  # Thickness of line

ax.legend(
    loc="center left", bbox_to_anchor=(1.0, 0.5)  # Placement of legend within bbox
)  # Location of boundary box (bbox)
plt.show()


In [ ]:
# Run time series decomposition on each time series of the DataFrame
df_names = df.columns
df_decomp = {ts: sm.tsa.seasonal_decompose(df[ts]) for ts in df.columns}

# Capture the seasonal, trend and noise components for the decomposition of each time series
seasonal_dict = {ts: df_decomp[ts].seasonal for ts in df_names}
trend_dict = {ts: df_decomp[ts].trend for ts in df_names}
noise_dict = {ts: df_decomp[ts].resid for ts in df_names}

# Create a DataFrame from the dictionaries
seasonality_df = pd.DataFrame.from_dict(seasonal_dict)
trend_df = pd.DataFrame.from_dict(trend_dict)
noise_df = pd.DataFrame.from_dict(noise_dict)

# Remove the label for the index
seasonality_df.index.name = None
trend_df.index.name = None
noise_df.index.name = None

In [ ]:
# Look at individual seasonality, trend or noise
noise_df["Agriculture"].plot()
# Change the dataframe and colum to explore


In [ ]:
# Create a faceted plot of the seasonality_df DataFrame
trend_df[["Agriculture", "Manufacturing"]].plot(
    subplots=True,  # Show multiple plots
    layout=(2, 1),  # Choose layout for showing plots
    sharey=False,  # Share the y axis
    legend=True,  # Show legend
    fontsize=10,  # Set fontsize
    linewidth=2,  # Set widht of line
)

plt.suptitle("Seasonality in Agriculture and Manufacturing", size=15)
plt.show()


In [ ]:
# Get correlation matrix of your chosen dataframe
seasonality_corr = seasonality_df.corr(
    method="spearman"
)  # Choose method to calculate correlation

# Customize the clustermap of the correlation matrix
fig = sns.clustermap(
    seasonality_corr,  # Choose correlation matrix to visualize
    annot=True,  # Show annotations
    annot_kws={"size": 10},  # Customize annotations
    linewidths=0.4,
    figsize=(15, 10),
)

plt.setp(
    fig.ax_heatmap.xaxis.get_majorticklabels(),
    rotation=90,  # Change rotation of x-labels
)
plt.show()
